# DBS/TMS awake/asleep comparison- Consciousness

## Load in data

This has to be converted to a fif file first using the bv2fif.py script

In [ ]:
import os,re,glob
import os.path as op
from MEEGbuddy import MEEGbuddy
from MEEGbuddy.bv2fif import bv2fif
import numpy as np
from mne import read_epochs
from mne.io import Raw
import matplotlib.pyplot as plt

subject = ###
session = ###
ref_ch=###
perturbation=###

stimuli = {perturbation:['Response/R 15',-0.5,1.0]}
baseline = ['Response/R 15',-0.4,0]

datarex = re.compile(r'[%s|%s]*.vhdr' %(subject.lower(),subject.upper()))
correx = re.compile(r'[%s|%s]*.bvct' %(subject.lower(),subject.upper()))
    
if True:
    files = glob.glob(op.join(os.getcwd(),'data',perturbation,subject,session,'*'))
    if not files:
        print('Subject data must be in a folder with their name')
    corf = None
    for f in files:
        if correx.search(f):
            corf = f
    for f in files:
        if datarex.search(f) and not 'avg' in f and not '-TMS.vhdr' == op.basename(f)[-9:] and not '-DBS.vhdr' == op.basename(f)[-9:]:
            fdata = (os.path.join(os.path.dirname(f),
                     op.basename(f).split('.')[0]+'-raw.fif'))
            epof = (os.path.join(os.path.dirname(f),
                    op.basename(f).split('.')[0]+'-epo.fif'))
            if not op.isfile(fdata) or not op.isfile(epof):
                bv2fif(f,corf,ref_ch=ref_ch,dbs=perturbation.lower()=='dbs')
            #
            fs_dir = op.join(os.getcwd(),'data',perturbation)
            bemf = op.join(fs_dir,subject,'bem','%s-5120-5120-5120-bem-sol.fif' %(subject))
            srcf = op.join(fs_dir,subject,'bem','%s-oct-6-src.fif' %(subject))
            transf = op.join(fs_dir,subject,session,'COR-%s_%s.fif' %(subject,session))
            #
            epochs = read_epochs(epof)
            print('Loading in to MEEGbuddy %s %s' %(subject,session))
            data = MEEGbuddy(subject=subject,session=session,fdata=fdata,baseline=baseline,stimuli=stimuli,task=perturbation+'-EEG',eeg=True,
                             subjects_dir=op.join(os.getcwd(),'data'),exclude_response=[0,len(epochs)-1],
                             epochs=epochs,event=pertubation,fs_subjects_dir=fs_dir,bemf=bemf,srcf=srcf,transf=transf)

Extracting parameters from ./JM/1_31_2018_JM_con_pilot.vhdr...
Setting channel info structure...
Reading 0 ... 99771499  =      0.000 ...  3990.860 secs...


### Mark Bads

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'))

for data in tqdm(data_struct):
    print(data.subject + ' ' + data.session)
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_raw(keyword='Preprocessed') and not data._has_PSD(keyword='ICA'):
        data.filterRaw(keyword_out='Preprocessed',l_freq=0.5,overwrite=True)
        data.makeEpochs(keyword_in='Preprocessed',keyword_out='Vis',overwrite=True)
        data.filterEpochs(event,keyword_in='Vis',h_freq=40,l_freq=None,overwrite=True) 
        data.plotEvoked(event,keyword='Vis',show=True,detrend=1,downsample=False,ylim={'eeg':[-100,100]})
        data.plotRaw(keyword_in='Preprocessed',l_freq=None,h_freq=None,overwrite=True)

### Choose and Apply Interpolation Raw data

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'))

for data in tqdm(data_struct):
    print(data.subject + ' ' + data.session)
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_raw(keyword='Interpolated') and not data._has_PSD(keyword='ICA'):
        raw_interp,_ = data.plotInterpolateArtifact('TMS',keyword='Preprocessed',mode='spline',npoint_art=11,offset=1,tmin=-0.1,tmax=0.1,ylim=[-1e-4,1e-4])
        plt.show()
        data.applyInterpolation(raw_interp,keyword_out='Interpolated',overwrite=True)

### Do ICA on Raw Data

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'))

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_ICA(keyword='ICA') and not data._has_PSD(keyword='ICA'):
        # precompute ICA
        data.findICA(keyword_in='Interpolated',keyword_out='ICA',overwrite=True,overwrite_ica=True)
        ok = True
        while ok:
            data.plotICA(keyword_in='Interpolated',keyword_out='ICA')
            ok = input('Press Enter to continue...')
        #
        data.makeEpochs(keyword_in='ICA',keyword_out='Vis',overwrite=True)
        data.filterEpochs(event,keyword_in='Vis',h_freq=40,overwrite=True)
        data.plotEvoked(event,keyword='Vis',ylim={'eeg':[-60,40]},tmin=-0.5,tmax=0.5)
        plt.show()

### Multitaper Spectrogram and Sleep/Wake Assignment

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'))

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_PSD(keyword='ICA'): 
        state_times = data.psdMultitaper(keyword='ICA',overwrite=False,ch='Oz',vmax=-10)
        data.assignConditionFromStateTimes('TMS',state_times)

### Clean Up Raw

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as pltdata

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'))

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if data._has_PSD(keyword='ICA'):
        data.remove(keyword='Preprocessed')
        data.remove(event,keyword='Vis')
        data.remove(keyword='Interpolated')

## Epochs

### Plot Epochs

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),)

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if (not data._has_epochs(event,keyword='Clean') and 
        not all([data._has_noreun_PCI(event,'State',value,keyword='Post-ICA') 
                for value in data._default_values('State')])):
        data.plotEvoked(event,show=True,detrend=1,downsample=False,ylim={'eeg':[-100,100]})
        data.plotEpochs(event,keyword_out='Clean',l_freq=None,h_freq=None,overwrite=True)

### Interpolate Artifact

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if (not data._has_epochs(event,keyword='Interpolated') and 
        not all([data._has_noreun_PCI(event,'State',value,keyword='Post-ICA') 
                for value in data._default_values('State')])):
        epo_interp,_ = data.plotInterpolateArtifact(event,use_raw=False,keyword='Clean',mode='spline',
                                                    npoint_art=11,offset=1,tmin=-0.1,tmax=0.1,ylim=[-6e-5,4e-5])
        plt.show()
        data.applyInterpolation(epo_interp,event,keyword_out='Interpolated',overwrite=True)

### Apply ICA to Epochs

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if (not data._has_epochs(event,keyword='ICA') and 
        not all([data._has_noreun_PCI(event,'State',value,keyword='Post-ICA') 
                for value in data._default_values('State')])):
        data.findICA(event=event,keyword_in='Interpolated',keyword_out='ICA')

### QC ICA

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_epochs(event,keyword='Post-ICA'):
        # Quality check ICA (will need to do multiple times if the components are hand-selected) (click on the component name to see topo)
        ok = True
        while ok:
            try:
                data.plotICA(event=event,keyword_in='Interpolated',keyword_out='ICA')
                data.filterEpochs(event,h_freq=40,keyword_in='ICA',keyword_out='Vis',overwrite=True)
                data.plotEvoked(event,show=True,detrend=1,keyword='Vis',ylim={'eeg':[-100,100]})
                data.plotTopo(event,condition='State',keyword='Vis',
                              show=True,detrend=1,downsample=False)
                data.plotEpochs(event,keyword_in='ICA',keyword_out='Post-ICA',overwrite=True)
                ok = input('Press ok to continue, type anything to go back')
                while ok:
                    data.filterEpochs(event,h_freq=40,keyword_in='Post-ICA',keyword_out='Vis',overwrite=True)
                    data.plotEvoked(event,show=True,detrend=1,keyword='Vis',ylim={'eeg':[-100,100]})
                    data.plotTopo(event,condition='State',keyword='Vis',
                                  show=True,detrend=1,downsample=False)
                    data.plotEpochs(event,keyword_in='Post-ICA',overwrite=True)
            except Exception as e:
                print(e)
                print(data.subject)
            ok = input('Press enter to move on, type anything to go back\n')

## Source Space

In [3]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not all([data._has_source(event,'State',value,keyword='Post-ICA') for value in data._default_values('State')]):
        epochs = data._load_epochs(event,keyword='Post-ICA')
        data._save_epochs(epochs.crop(tmin=-0.5,tmax=0),'Baseline',keyword='Post-ICA')
        data.epochs2source(event,'State',keyword_in='Post-ICA',overwrite=True)



  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 251.45it/s]

### Plot Source Space (optional)

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    data.plotSourceSpace(event,'State',keyword='ICA',tmin=-0.1,tmax=0.5)

## PCI

The interesting part!

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not all([data._has_noreun_PCI(event,'State',value,keyword='Post-ICA') 
                for value in data._default_values('State') if not value == 'Neither']):
        data.noreunPhi('TMS','State',keyword_in='Post-ICA',npoint_art=10,
                       tmin=0.0,tmax=0.3,shared_baseline=False,
                       bl_tmin=-0.4,bl_tmax=-0.01,downsample=False,
                       recalculate_PCI=True,recalculate_baseline=True)

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    try:
        data.plotNoreunPCI('TMS','State',keyword='Post-ICA')
        plt.show()
    except:
        pass

## Wavelet Connectivity Visualization (not part of pipeline)

In [ ]:
import matplotlib.pyplot as plt
from MEEGbuddy import loadMEEGbuddies
import os
import os.path as op
from tqdm import tqdm

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in data_struct:
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    data.waveletConnectivity(event,'State',keyword_in='Clean',
                             keyword_out='WC',downsample=False,
                             bands={'alpha':(8,15),'beta':(15,35),
                                    'gamma':(35,150)},tmin=-0.5,tmax=1,
                             time_scalar=2,vis_tmin=-0.1,vis_tmax=0.2,
                             threshold=0.5)
    break